In [ ]:
import numpy as np

from Related_Codes.DCNAutoencoder import DCNAutoencoder, DCNCDAutoencoder
from Related_Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding, Isomap, TSNE

import torch

import os
import sys

from Datasets.Datasets_Functions import *
from Visualization import Visualization

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
 

In [ ]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
print(hashmap)
dataset_name = "emnist_balanced_digits"
dataset_properties = hashmap[dataset_name]
batch_size = dataset_properties['batch_size'] = 256
n_clusters = dataset_properties['n_clusters'] = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)
print(input_dim)

In [ ]:
visualization = Visualization()

### Dec

In [ ]:
n_clusters = dataset_properties['n_clusters']

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size']

# Pre-Training Epochs and Learning Rate
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
n_epochs = 100
lr = 5e-4
alpha = 1.0
momentum = 0.9
latent_dim = 10
n_channels = 1
is_MLP_AE = False

if(is_MLP_AE):
    input_dim = 784
    dec_autoencoder = DECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim)
else:  
    input_dim = 1
    dec_autoencoder = DECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, n_channels=n_channels) 
dec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha )
dec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dec_autoencoder.set_path()
dec_autoencoder = dec_autoencoder.to(device)

pretrain = True
if(pretrain):
    dec_autoencoder.pretrain_autoencoder()
    dec_autoencoder.save_pretrained_weights()
else:
    model_save_path = dec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dec_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
dec_autoencoder.kmeans_initialization()
dec_autoencoder.train_autoencoder()
cluster_centers = dec_autoencoder.get_cluster_centers()
_, dec_autoencoder_reduced_data, labels = dec_autoencoder.get_latent_data()
General_Functions().save_excel(dec_autoencoder.data_dir_path, dec_autoencoder.df_eval)
#visualization.plot(dec_autoencoder_reduced_data, labels, labels, cluster_centers, dec_autoencoder.data_dir_path)
visualization.plot_tsne(dec_autoencoder_reduced_data, labels, labels, cluster_centers, dec_autoencoder.data_dir_path)